## Importing Libraries

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout,Flatten,Dense,Activation,BatchNormalization, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Image Properties

In [2]:
Image_Width=128
Image_Height=128
Image_Size=(Image_Width,Image_Height)
Image_Channels=3

# I have taken Image Channel as 3 because the images are coloured images (RGB format), Not Grayscale

## Prepare dataset for training model

In [3]:
filenames=os.listdir("./train")
categories=[]

for f_name in filenames:
    category=f_name.split('.')[0]
    if category=='dog':
        categories.append(1)
    else:
        categories.append(0)

# I am marking every 'dog' image as 1 and 'cat' image as 0
# Dog - 1     Cat - 0

df=pd.DataFrame({
    'filename':filenames,
    'category':categories
})

# I created a Pandas Dataframe which keeps the values of 'filenames' from the train folder and their categories 
#   which was stored in the for loop (Dog as 1 & cat as 0)

## Create the neural net model

In [4]:
model=Sequential()

model.add(Input(shape=(Image_Width, Image_Height, Image_Channels)))

model.add(Conv2D(32,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


# Sequential - It is used to create a sequential model in Keras, which is a linear stack of layers. It allows us to 
#              easily build a model by adding layers one after another in a sequential manner.

# Conv2D - It is a 2D convolutional layer commonly used in convolutional neural networks (CNNs) for image processing 
#          tasks. Convolutional layers apply a set of learnable filters to the input image, capturing spatial patterns 
#          and features.

# MaxPooling2D - It is a pooling layer used to down-sample the spatial dimensions of the feature maps generated by 
#                convolutional layers. It helps to reduce the spatial size, extract dominant features, and introduce 
#                translation invariance.

# Dropout - It is a regularization technique used to prevent overfitting in neural networks. It randomly sets a fraction 
#           of input units to zero during training, which helps to reduce interdependent learning and improves 
#           generalization.

# Flatten - is a layer that converts a multi-dimensional input (such as feature maps from convolutional layers) into 
#           a one-dimensional vector. It is used as a transition layer between convolutional layers and fully 
#           connected layers.

# Dense - It is a fully connected layer where each neuron is connected to every neuron in the previous layer. It is the
#         part of ANN. It is commonly used in the final stages of a neural network for classification or regression tasks.

# BatchNormalization - It is a technique used to normalize the activations of the previous layer at each batch during 
#                      training. It helps to stabilize the learning process, reduce the sensitivity to the choice of 
#                      initialization, and allow higher learning rates.

# Q. Why the number of layers are in power of 2 ?? 
#    Modern computer architectures, including CPUs and GPUs, are optimized for binary arithmetic
#    When training deep learning models on distributed systems or using model parallelism, using powers of 2 can simplify 
#         the partitioning and distribution of workload across multiple devices.

# Q. Why the no. of layers in Conv2D always increases gradually (here 32->64->128) ?? Why not constant or decrease ??
#    In the initial layers of the network, the Conv2D layers with a smaller number of filters (e.g., 32) are used to 
#    capture low-level features such as edges, corners, and simple patterns in the input image. These low-level features 
#    are fundamental building blocks for higher-level features. As the network goes deeper, the Conv2D layers with an 
#    increasing number of filters (e.g., 64, 128) are used to learn more complex and abstract features. These layers 
#    combine and transform the low-level features learned in the previous layers to capture higher-level concepts and 
#    patterns in the image.

In [5]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,942,786 (49.37 MB)

 Trainable params: 12,941,314 (49.37 MB)

 Non-trainable params: 1,472 (5.75 KB)

## Define callbacks and learning rate

In [6]:
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',patience = 2,
                                            verbose = 1,factor = 0.5,min_lr = 0.00001)
callbacks = [earlystop,learning_rate_reduction]

# EarlyStopping(patience = 10) - The patience parameter is set to 10, which means that the training will stop if there is 
#                                no improvement in the monitored metric (validation loss or accuracy) for 10 consecutive 
#                                epochs.

# learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',patience = 2,
#                                             verbose = 1,factor = 0.5,min_lr = 0.00001) 
#                           - The monitor parameter is set to 'val_accuracy', which means that the callback will monitor 
#                             the validation accuracy metric.
#                           - The patience parameter is set to 2, indicating that the learning rate will be reduced if 
#                             there is no improvement in the monitored metric for 2 consecutive epochs.
#                           - The verbose parameter is set to 1, which enables verbose mode, providing more detailed 
#                             output during training.
#                           - The factor parameter is set to 0.5, meaning that when the learning rate is reduced, it will 
#                             be multiplied by 0.5 (i.e., halved).
#                           - The min_lr parameter is set to 0.00001, specifying the lower bound for the learning rate. 
#                             The learning rate will not be reduced below this value.

## Manage data

In [7]:
df["category"] = df["category"].replace({0:'cat',1:'dog'})
train_df,validate_df = train_test_split(df, test_size=0.20, random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

total_train=train_df.shape[0]
total_validate=validate_df.shape[0]

batch_size=10

## Training and validation data generator

In [8]:
train_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1
                                )
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                 "./train/",x_col='filename',y_col='category',
                                                 target_size=Image_Size,
                                                 class_mode='categorical',
                                                 batch_size=batch_size)


validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "./train/", 
    x_col='filename',
    y_col='category',
    target_size=Image_Size,
    class_mode='categorical',
    batch_size=batch_size
)

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.


## Model Training

In [9]:
epochs=10
history = model.fit(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)

Epoch 1/10


/Users/susovanpatra/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1333/1333 ━━━━━━━━━━━━━━━━━━━━ 480s 358ms/step - accuracy: 0.5765 - loss: 0.9216 - val_accuracy: 0.5986 - val_loss: 0.7104 - learning_rate: 0.0010
Epoch 2/10
1333/1333 ━━━━━━━━━━━━━━━━━━━━ 0s 102us/step - accuracy: 0.5333 - loss: 0.7693 - val_accuracy: 0.6000 - val_loss: 0.6664 - learning_rate: 0.0010
Epoch 3/10


2024-04-29 20:45:34.954654: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/susovanpatra/anaconda3/lib/python3.11/contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-04-29 20:45:35.068029: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


1333/1333 ━━━━━━━━━━━━━━━━━━━━ 508s 380ms/step - accuracy: 0.6412 - loss: 0.6461 - val_accuracy: 0.7183 - val_loss: 0.5493 - learning_rate: 0.0010
Epoch 4/10
1333/1333 ━━━━━━━━━━━━━━━━━━━━ 1s 96us/step - accuracy: 0.6667 - loss: 0.4784 - val_accuracy: 0.8000 - val_loss: 0.5075 - learning_rate: 0.0010
Epoch 5/10


...

2024-04-29 21:01:20.678801: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-04-29 21:01:20.785009: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


1333/1333 ━━━━━━━━━━━━━━━━━━━━ 409s 306ms/step - accuracy: 0.7256 - loss: 0.5401 - val_accuracy: 0.6887 - val_loss: 0.6618 - learning_rate: 5.0000e-04
Epoch 8/10
1333/1333 ━━━━━━━━━━━━━━━━━━━━ 0s 86us/step - accuracy: 0.7333 - loss: 0.4125 - val_accuracy: 1.0000 - val_loss: 0.1464 - learning_rate: 5.0000e-04
Epoch 9/10


2024-04-29 21:08:09.672910: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-04-29 21:08:09.774061: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


1333/1333 ━━━━━━━━━━━━━━━━━━━━ 405s 304ms/step - accuracy: 0.7508 - loss: 0.5017 - val_accuracy: 0.7906 - val_loss: 0.4470 - learning_rate: 5.0000e-04
Epoch 10/10
   1/1333 ━━━━━━━━━━━━━━━━━━━━ 7:26 335ms/step - accuracy: 0.7333 - loss: 0.5109
Epoch 10: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
1333/1333 ━━━━━━━━━━━━━━━━━━━━ 0s 36us/step - accuracy: 0.7333 - loss: 0.5109 - val_accuracy: 1.0000 - val_loss: 0.1219 - learning_rate: 5.0000e-04


2024-04-29 21:14:55.210055: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-04-29 21:14:55.237691: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
